In [1]:
import sys
sys.path.append("/home/code")

### 데이터 셋 테스트

In [1]:
from database import *

datasetdb = DatasetDB()
datasetdb.get_all_id()

['ai_hub_korean_det',
 'ai_hub_korean_rec',
 'ICDAR2015',
 'outsourcing1(exception)_std',
 'outsourcing1(exception)_str',
 'outsourcing1_std',
 'outsourcing1_str',
 'sangmu_std_DH',
 'sangmu_std_DI',
 'sangmu_std_MH',
 'sangmu_str_DH',
 'sangmu_str_DI',
 'sangmu_str_MH',
 'test']

### 레이블 셋 테스트

In [6]:
from database import *
datadb = DatasetDB()
labeldb = LabelsetDB()
datadb.get_all_id()

# labeldb.make("test", ["test"], split_ratio=[8, 1, 1], random_seed=100)
labeldb.get_config("test", relative_to = "project")["label"]

{'eval': ['labelsets/test/eval_label.txt'],
 'test': ['labelsets/test/test_label.txt'],
 'train': ['labelsets/test/train_label.txt']}

In [1]:
from database import *
datadb = DatasetDB()
datadb.relative_to("id", "path", "absolute")


'E:/workspace/paddleocr/datasets/id/path'

In [ ]:
from pathlib import Path
x = "./hello/world/text.txt"
Path("./test/")/x

WindowsPath('test/hello/world/text.txt')

#### 레이블 셋 생성

In [7]:
from database import *
datadb = DatasetDB()
labeldb = LabelsetDB()
datadb.get_all_id()

labeldb.make("sangmu_std", ["sangmu_std_MH", "sangmu_std_DI", "sangmu_std_DH"], split_ratio=[8, 1, 1], random_seed=100)
labeldb.make("sangmu_str", ["sangmu_str_MH", "sangmu_str_DI", "sangmu_str_DH"], split_ratio=[8, 1, 1], random_seed=100)
labeldb.make("outsourcing_std", ["outsourcing1_std", "outsourcing1(exception)_std"], [8, 1, 1], random_seed=100)
labeldb.make("outsourcing_str", ["outsourcing1_str", "outsourcing1(exception)_str"], [8, 1, 1], random_seed=100)

labeldb.make("sangmu_and_outsourcing_std", ["outsourcing1_std", "outsourcing1(exception)_std"], [8, 2, 0], individual_split_ratio={
    "sangmu_std_MH":[0, 0, 1],
    "sangmu_std_DI":[0, 0, 1],
    "sangmu_std_DH":[0, 0, 1]
    }, random_seed=100)

labeldb.make("sangmu_and_outsourcing_str", ["outsourcing1_str", "outsourcing1(exception)_str"], [8, 2, 0], individual_split_ratio={
    "sangmu_str_MH":[0, 0, 1],
    "sangmu_str_DI":[0, 0, 1],
    "sangmu_str_DH":[0, 0, 1]
    }, random_seed=100)

2. save label files
3. make infer label files
2. save label files
3. make infer label files


#### k_fold 레이블 셋 생성

In [ ]:
# from database import *
# labeldb = LabelsetDB()
# labeldb.make_k_fold(["test2"], "test_k_fold", k=5)
# labeldb.get_config("test_k_fold_5_1")

##### Model

In [1]:
from database import *
modeldb = ModelDB()
import os
# modeldb.get_config("en_PP-OCRv3_det")
modeldb.get_all_id()
modeldb.get_config("en_PP-OCRv3_det", relative_to="absolute")
# for id in modeldb.get_all_id():
#     print(id)
#     modeldb.make_inference_model(id)

{'inference_model': 'E:/workspace/paddleocr/models/en_PP-OCRv3_det/inference_model/Student/inference.pdmodel',
 'inference_model_dir': 'E:/workspace/paddleocr/models/en_PP-OCRv3_det/inference_model/Student',
 'inference_model_weight': 'E:/workspace/paddleocr/models/en_PP-OCRv3_det/inference_model/Student/inference.pdiparams',
 'pretrained_model_dir': 'E:/workspace/paddleocr/models/en_PP-OCRv3_det/pretrained_model',
 'pretrained_model_weight': 'E:/workspace/paddleocr/models/en_PP-OCRv3_det/pretrained_model/pretrained.pdparams',
 'train_config': 'E:/workspace/paddleocr/models/en_PP-OCRv3_det/pretrained_model/config.yml'}

# Work

#### Work make

In [2]:
from database import *
workdb = WorkDB()
# workdb.make("det_PPOCR_sangmu", ["sangmu_std"], "ml_PP-OCRv3_det")
workdb.make("det_PPOCR_outsourcing", ["outsourcing_std"], "ml_PP-OCRv3_det")
# workdb.make("det_PPOCR_sangmu_and_outsourcing", ["sangmu_and_outsourcing_std"], "ml_PP-OCRv3_det")

# workdb.make("rec_PPOCR_sangmu", ["sangmu_str"], "korean_PP-OCRv3_rec")
# workdb.make("rec_PPOCR_outsourcing", ["outsourcing_str"], "korean_PP-OCRv3_rec")
# workdb.make("rec_PPOCR_sangmu_and_outsourcing", ["sangmu_and_outsourcing_str"], "korean_PP-OCRv3_rec")

#### Work Infer

In [ ]:
from database import *
workdb = WorkDB()
workdb.infer("det_PPOCR_sangmu", "best", "test")
workdb.infer("det_PPOCR_outsourcing", "best", "test")
workdb.infer("det_PPOCR_sangmu_and_outsourcing", "latest", "test")

#### Work Train

In [14]:
from database import *
workdb = WorkDB()
modeldb = ModelDB()
labelsetdb = LabelsetDB()


workdb.train("rec_PPOCR_sangmu", "latest", 10, relative_to="project")
workdb.train("rec_PPOCR_outsourcing", "latest", 1, relative_to="project")
workdb.train("rec_PPOCR_sangmu_and_outsourcing", "latest", 10, relative_to="project")

works = ["det_PPOCR_sangmu", "det_PPOCR_outsourcing", "rec_PPOCR_sangmu_and_outsourcing"]
tasks = ["train", "eval", "test"]
for work in works:
    workdb.train(work, "latest", 100, relative_to="absolute")

python /home/code/PaddleOCR/tools/train.py -c works/det_PPOCR_sangmu/train_config.yml -o Global.checkpoints=/home/works/det_PPOCR_sangmu/trained_model/latest Global.epoch_num=100 Global.save_model_dir=works/det_PPOCR_sangmu/trained_model Train.dataset.data_dir=./datasets Train.dataset.label_file_list=['labelsets/sangmu_std/train_label.txt'] Eval.dataset.data_dir=./datasets Eval.dataset.label_file_list=['labelsets/sangmu_std/eval_label.txt']
python /home/code/PaddleOCR/tools/train.py -c works/det_PPOCR_outsourcing/train_config.yml -o Global.checkpoints=/home/works/det_PPOCR_outsourcing/trained_model/latest Global.epoch_num=100 Global.save_model_dir=works/det_PPOCR_outsourcing/trained_model Train.dataset.data_dir=./datasets Train.dataset.label_file_list=['labelsets/outsourcing_std/train_label.txt'] Eval.dataset.data_dir=./datasets Eval.dataset.label_file_list=['labelsets/outsourcing_std/eval_label.txt']
python /home/code/PaddleOCR/tools/train.py -c works/rec_PPOCR_sangmu_and_outsourcing/

#### Work Eval

In [1]:
from database import *
workdb = WorkDB()
works = ["rec_PPOCR_sangmu", "rec_PPOCR_outsourcing", "rec_PPOCR_sangmu_and_outsourcing"]
works = ["det_PPOCR_sangmu", "det_PPOCR_outsourcing", "det_PPOCR_sangmu_and_outsourcing"]
tasks = ["train", "eval", "test"]
epoch = range(1, 201, 1)

works = ["det_PPOCR_sangmu", "det_PPOCR_outsourcing"]
tasks = ["train", "eval", "test"]

epochs = range(1, 101, 1)
works = ["det_PPOCR_sangmu"]
for work in works:
    for epoch in epochs:
        for task in tasks:
            workdb.eval(work, version=epoch, task=task, relative_to="absolute")

works = ["det_PPOCR_outsourcing"]
epochs = range(1, 201, 1)
for work in works:
    for epoch in epochs:
        for task in tasks:
            workdb.eval(work, version=epoch, task=task, relative_to="absolute")
            
works = ["det_PPOCR_sangmu_and_outsourcing"]
epochs = range(1, 51, 1)
for work in works:
    for epoch in epochs:
        for task in tasks:
            workdb.eval(work, version=epoch, task=task, relative_to="absolute")

# workdb.eval("det_PPOCR_oursourcing", "best", 10, relative_to="project") 
# workdb.eval("det_PPOCR_sangmu_and_oursourcing", "best", 10)

(id:det_PPOCR_sangmu, version:1, task:train) already evaluated
(id:det_PPOCR_sangmu, version:1, task:eval) already evaluated
(id:det_PPOCR_sangmu, version:1, task:test) already evaluated
(id:det_PPOCR_sangmu, version:2, task:train) already evaluated
(id:det_PPOCR_sangmu, version:2, task:eval) already evaluated
(id:det_PPOCR_sangmu, version:2, task:test) already evaluated
(id:det_PPOCR_sangmu, version:3, task:train) already evaluated
(id:det_PPOCR_sangmu, version:3, task:eval) already evaluated
(id:det_PPOCR_sangmu, version:3, task:test) already evaluated
(id:det_PPOCR_sangmu, version:4, task:train) already evaluated
(id:det_PPOCR_sangmu, version:4, task:eval) already evaluated
(id:det_PPOCR_sangmu, version:4, task:test) already evaluated
(id:det_PPOCR_sangmu, version:5, task:train) already evaluated
(id:det_PPOCR_sangmu, version:5, task:eval) already evaluated
(id:det_PPOCR_sangmu, version:5, task:test) already evaluated
(id:det_PPOCR_sangmu, version:6, task:train) already evaluated
(i

AssertionError: version should be 'best', 'latest', 'pretrained', or positive integer less than trained_epoch 50 but 51 is given

### Make Inference Model

In [1]:
from database import *
workdb = WorkDB()
workdb.export("det_PPOCR_sangmu", "best")
workdb.export("rec_PPOCR_sangmu", "best")

python code/PaddleOCR/tools/export_model.py -c /home/works/det_PPOCR_sangmu/trained_model/config.yml -o Global.save_inference_dir=/home/works/det_PPOCR_sangmu/inference_models/inference_best Global.checkpoints=/home/works/det_PPOCR_sangmu/trained_model/best_model/model.pdparams
python code/PaddleOCR/tools/export_model.py -c /home/works/rec_PPOCR_sangmu/trained_model/config.yml -o Global.save_inference_dir=/home/works/rec_PPOCR_sangmu/inference_models/inference_best Global.checkpoints=/home/works/rec_PPOCR_sangmu/trained_model/best_model/model.pdparams


'/home/works/det_PPOCR_sangmu/inference_models/inference_best'

In [2]:
from database import *
workdb = WorkDB()
workdb.get_report_df("det_PPOCR_sangmu_old")

,version,step,acc,loss,precision,recall,dataset
0,latest,NaN,NaN,NaN,0.666667,1.000000,NaN
0,latest,NaN,NaN,NaN,0.666667,1.000000,NaN
0,latest,NaN,NaN,NaN,0.399777,0.656307,test


In [ ]:
from database import *
import project
from pathlib import Path
# config = "configs\det\det_mv3_db.yml"
# labelsets = ["test", "test_k_fold_5_1"]
# model = "MobileNetV3_large_x0_5"
# name = "test"
# WorkDB().make(name, labelsets, model, config)

WorkDB().train("test", 30)


In [5]:

from database import *
from pathlib import Path
import platform

def full_infer(image_dir, det_model_dir, rec_model_dir, save_dir):
    code = 'code/PaddleOCR/tools/infer/predict_system.py'
    det_algorithm="DB"

    # rec_model_dir = "E:/workspace/paddleocr/outputs/rec___korean_PP-OCRv3_rec___rec_08_02_90_random_k_fold_5_1___default_config___finetune/trained_model/inference_model"
    rec_char_dict_path = f"{project.PROJECT_ROOT}/code/PaddleOCR/ppocr/utils/dict/korean_dict.txt"
    os_name = platform.system()
    if os_name == "Windows":
        vis_font_path = "C:\Windows\Fonts\malgun.ttf"
    elif os_name == "Linux":
        vis_font_path = "/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf"

    command = f"""python {code} --image_dir={image_dir} \
    --det_model_dir={det_model_dir} \
    --det_algorithm={det_algorithm} \
    --rec_model_dir={rec_model_dir} \
    --rec_image_shape="3, 32, 100" \
    --rec_char_dict_path={rec_char_dict_path} \
    --vis_font_path={vis_font_path} \
    --draw_img_save_dir={save_dir} \
    --drop_score=0.3 \
    --det_db_box_thresh=0.4"""
    
    path = Path(project.PROJECT_ROOT)/"inference.sh"
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "a") as f:
        f.write(command+"\n")
    
modeldb = ModelDB()
workdb = WorkDB()

# det_model_dir = modeldb.get_config("MobileNetV3_large_x0_5")["inference_model_dir"] 
# rec_model_dir = modeldb.get_config("korean_PP-OCRv3_rec")["inference_model_dir"]
# save_dir = f"{project.PROJECT_ROOT}/inference/Mobile_PaddleOCR_pretrained"
# full_infer(det_model_dir, rec_model_dir, save_dir)

# image_dir = "./origin_datasets/sangmu_real_image_det(cropped)"
# det_model_dir = modeldb.get_config("ml_PP-OCRv3_det")["inference_model_dir"]
# rec_model_dir = modeldb.get_config("korean_PP-OCRv3_rec")["inference_model_dir"]
# save_dir = f"{project.PROJECT_ROOT}/inference/PaddleOCR_PaddleOCR_pretrained_sangmu"
# full_infer(image_dir, det_model_dir, rec_model_dir, save_dir)

# image_dir = "./origin_datasets/sangmu_real_image_det(cropped)"
# det_model_dir = workdb.get_config("det___ml_PP-OCRv3_det___ai_hub_det_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
# rec_model_dir = workdb.get_config("rec___korean_PP-OCRv3_rec___rec_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
# save_dir = f"{project.PROJECT_ROOT}/inference/PPOCR_PPOCR_tuned_sangmu"
# full_infer(image_dir, det_model_dir, rec_model_dir, save_dir)

# image_dir = "./datasets/ai_hub_korean_det_partial_test/images"
# det_model_dir = workdb.get_config("det___ml_PP-OCRv3_det___ai_hub_det_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
# rec_model_dir = workdb.get_config("rec___korean_PP-OCRv3_rec___rec_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
# save_dir = f"{project.PROJECT_ROOT}/inference/PPOCR_PPOCR_tuned_aihub"
# full_infer(image_dir, det_model_dir, rec_model_dir, save_dir)

from database import *
workdb = WorkDB()


image_dir = "/./datasets/ai_hub_korean_det_partial_test/images"
image_dir = f"{project.PROJECT_ROOT}/resources/test_images"

det_model_dir = workdb.get_inference_model("det_PPOCR_sangmu", "best", relative_to="absolute")
det_model_dir = f"{det_model_dir}/Student"

rec_model_dir = workdb.get_inference_model("rec_PPOCR_sangmu", "best", relative_to="absolute")
# det_model_dir = f"{det_model_dir}/inference"

save_dir = f"{project.PROJECT_ROOT}/resources/result"
full_infer(image_dir, det_model_dir, rec_model_dir, save_dir)

In [ ]:
from pathlib import Path

Path("./datasets/ai_hub_korean_image_det/train2/source/간판/돌출간판/돌출간판1/간판_돌출간판_001449.jpg").exists()

False

In [ ]:
with open("E:/workspace/paddleocr/datasets/ai_hub_korean_det/label2.txt") as f:
    lines = [line.rstrip("\n") for line in f.readlines()]

a, b, c = 0, 0, 0
for line in lines:
    if "가로형간판" in line:
        a+= 1
    elif "세로형간판" in line:
        b+= 1
    elif "돌출간판" in line:
        c+= 1
print(a, b, c)

80242 25621 55803


In [ ]:
with open("E:/workspace/paddleocr/datasets/ai_hub_korean_det/label3.txt", encoding="cp949") as f:
    with open("E:/workspace/paddleocr/datasets/ai_hub_korean_det/label2.txt", "w", encoding="utf-8") as f2:
        for line in f.readlines():
            f2.write(line)

In [ ]:
from dataset.aihub.hangul_real_image_dataset import *
dataset = HangulRealImageDataset()


*******Checked Dir List*******
datasets\ai_hub_korean_det\train1\source\간판\세로형간판
datasets\ai_hub_korean_det\val1\source\간판\세로형간판
datasets\ai_hub_korean_det\train2\source\간판\가로형간판\가로형간판1
datasets\ai_hub_korean_det\train2\source\간판\돌출간판\돌출간판1
datasets\ai_hub_korean_det\train2\source\간판\세로형간판
datasets\ai_hub_korean_det\val2\source\간판\세로형간판
******************************


In [ ]:
dataset = HangulRealImage_BoxDetectionDataset(dataset)

In [ ]:
x, y = dataset[0]
dataset.show_xy(x, y)
y

[{'bbox': [[762, 306], [849, 306], [849, 518], [762, 518]], 'label': '소망'},
 {'bbox': [[754, 547], [837, 547], [837, 900], [754, 900]], 'label': '미용실'},
 {'bbox': [[730, 1053], [845, 1053], [845, 1147], [730, 1147]],
  'label': 'xxx'},
 {'bbox': [[871, 1078], [1047, 1078], [1047, 1139], [871, 1139]],
  'label': 'xxx'},
 {'bbox': [[1177, 732], [1479, 732], [1479, 794], [1177, 794]],
  'label': 'xxx'},
 {'bbox': [[1429, 707], [1502, 707], [1502, 748], [1429, 748]],
  'label': 'xxx'},
 {'bbox': [[1342, 1075], [1428, 1075], [1428, 1160], [1342, 1160]],
  'label': 'xxx'},
 {'bbox': [[184, 345], [361, 345], [361, 611], [184, 611]], 'label': 'xxx'},
 {'bbox': [[121, 291], [184, 291], [184, 342], [121, 342]], 'label': 'xxx'},
 {'bbox': [[365, 194], [460, 194], [460, 252], [365, 252]], 'label': 'xxx'},
 {'bbox': [[391, 536], [443, 536], [443, 587], [391, 587]], 'label': 'xxx'},
 {'bbox': [[355, 610], [517, 610], [517, 684], [355, 684]], 'label': 'xxx'},
 {'bbox': [[172, 748], [240, 748], [240, 

In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame({"epoch":[], "step":[], "acc":[], "loss":[], "precision":[], "recall":[]})

df = df._append({"epoch":1, "step":2, "acc":3, "loss":4, "precision":5, "recall":1}, ignore_index=True)
df.to_csv("test.csv")
df

,epoch,step,acc,loss,precision,recall
0,1.0,2.0,3.0,4.0,5.0,1.0


In [ ]:
df = pd.read_csv("test.csv", index_col=0)
df

,epoch,step,acc,loss,precision,recall
0,1.0,2.0,3.0,4.0,5.0,1.0


In [ ]:
from database import *

WorkDB().get_all_id()
WorkDB().get_config("det_MobileNet_sangmu")

{'id': 'det_MobileNet_sangmu',
 'labelsets': ['sangmu'],
 'model': 'MobileNetV3_large_x0_5',
 'pretrained': True,
 'train_config': './works/det_MobileNet_sangmu/train_config.yml',
 'result_path': './works/det_MobileNet_sangmu/result.csv'}

In [ ]:
from database import *

WorkDB().get_all_id()
# WorkDB().infer_det("det_MobileNet_sangmu")
# WorkDB().get_best_model_weight("det_MobileNet_sangmu")
# WorkDB().infer_det("det_MobileNet_sangmu", "test", "best")
WorkDB().infer_det("det_PPOCR_sangmu", "best", "test")